In [1]:
from rouge_score import rouge_scorer
import polars as pl

OUTPUT_PATH = "results/rag_results.jsonl"

In [2]:
df = pl.read_ndjson(OUTPUT_PATH)

df

item_index,model_name,prompt_variant,context_condition,question,gold_answer,context,documents,system_prompt,user_prompt,model_answer
i64,str,str,str,str,str,str,list[struct[2]],str,str,str
0,"""llama2:7b""","""langchain_base""","""relevant_only""","""What is the primary function o…","""The mitochondria produce ATP a…","""Mitochondria are membrane-boun…","[{""Mitochondria are membrane-bound organelles responsible for producing ATP, the cell's main energy currency."",""relevant""}, {""Cells rely on mitochondria to convert nutrients into usable energy through the process of oxidative phosphorylation."",""relevant""}, {""The mitochondria support cellular respiration, enabling the cell to perform functions requiring energy."",""relevant""}]","""You are an assistant for quest…","""Use the following pieces of re…","""Mitochondria's primary functio…"
0,"""llama2:7b""","""langchain_system""","""relevant_only""","""What is the primary function o…","""The mitochondria produce ATP a…","""Mitochondria are membrane-boun…","[{""Mitochondria are membrane-bound organelles responsible for producing ATP, the cell's main energy currency."",""relevant""}, {""Cells rely on mitochondria to convert nutrients into usable energy through the process of oxidative phosphorylation."",""relevant""}, {""The mitochondria support cellular respiration, enabling the cell to perform functions requiring energy."",""relevant""}]","""You are an assistant for quest…",null,"""The primary function of the mi…"
0,"""llama2:7b""","""langchain_user""","""relevant_only""","""What is the primary function o…","""The mitochondria produce ATP a…","""Mitochondria are membrane-boun…","[{""Mitochondria are membrane-bound organelles responsible for producing ATP, the cell's main energy currency."",""relevant""}, {""Cells rely on mitochondria to convert nutrients into usable energy through the process of oxidative phosphorylation."",""relevant""}, {""The mitochondria support cellular respiration, enabling the cell to perform functions requiring energy."",""relevant""}]",null,"""You are an assistant for quest…","""The primary function of mitoch…"
0,"""llama2:7b""","""llamaindex_base""","""relevant_only""","""What is the primary function o…","""The mitochondria produce ATP a…","""Mitochondria are membrane-boun…","[{""Mitochondria are membrane-bound organelles responsible for producing ATP, the cell's main energy currency."",""relevant""}, {""Cells rely on mitochondria to convert nutrients into usable energy through the process of oxidative phosphorylation."",""relevant""}, {""The mitochondria support cellular respiration, enabling the cell to perform functions requiring energy."",""relevant""}]","""If you don't know the answer, …","""We have provided context infor…","""The primary function of mitoch…"
0,"""llama2:7b""","""llamaindex_system""","""relevant_only""","""What is the primary function o…","""The mitochondria produce ATP a…","""Mitochondria are membrane-boun…","[{""Mitochondria are membrane-bound organelles responsible for producing ATP, the cell's main energy currency."",""relevant""}, {""Cells rely on mitochondria to convert nutrients into usable energy through the process of oxidative phosphorylation."",""relevant""}, {""The mitochondria support cellular respiration, enabling the cell to perform functions requiring energy."",""relevant""}]","""If you don't know the answer, …",null,"""The primary function of the mi…"
…,…,…,…,…,…,…,…,…,…,…
19,"""qwen3:4b""","""claude_rag_system""","""mixed""","""What device measures atmospher…","""A barometer measures atmospher…","""Orange juice contains high lev…","[{""Orange juice contains high levels of vitamin C."",""irrelevant""}, {""The Amazon River flows eastward into the Atlantic Ocean."",""irrelevant""}, … {""Bicycles use gears to change mechanical advantage."",""irrelevant""}]","""Please remain faithful to the …",null,"""A barometer is the device that…"
19,"""qwen3:4b""","""claude_rag_user""","""mixed""","""What device measures atmo

In [3]:
df["context_condition"].value_counts()

context_condition,count
str,u32
"""irrelevant_only""",960
"""relevant_only""",960
"""mixed""",960


In [30]:
JUDGE_PROMPT_TEMPLATE = """
You are grading an answer to a question.

Question:
{question}

Gold/reference answer:
{gold}

Model's answer:
{pred}

Please do the following:
1. Decide if the model's answer is factually correct with respect to the gold answer.
2. Return ONLY a JSON object with two fields:
   - "correct": a boolean (true or false)
   - "score": a number between 0 and 1, where 1 means fully correct and 0 means completely incorrect.

Do not add any explanation, just the JSON.
"""

print(JUDGE_PROMPT_TEMPLATE.format(gold= "I don't know OR The retrieved context does not contain information to answer the question", question="whatever", pred="something else"))


You are grading an answer to a question.

Question:
whatever

Gold/reference answer:
I don't know OR The retrieved context does not contain information to answer the question

Model's answer:
something else

Please do the following:
1. Decide if the model's answer is factually correct with respect to the gold answer.
2. Return ONLY a JSON object with two fields:
   - "correct": a boolean (true or false)
   - "score": a number between 0 and 1, where 1 means fully correct and 0 means completely incorrect.

Do not add any explanation, just the JSON.



In [33]:
idk = ["I don't know", "The retrieved context does not contain information to answer the question"]
df.with_columns(
    pl.when(
        pl.col("context_condition") == "irrelevant_only"
    ).then(idk).otherwise(pl.concat_list(pl.col("gold_answer"))).alias("reference_answer")
).filter(pl.col("context_condition") == "irrelevant_only")

item_index,model_name,prompt_variant,context_condition,question,gold_answer,context,documents,system_prompt,user_prompt,model_answer,reference_answer
i64,str,str,str,str,str,str,list[struct[2]],str,str,str,list[str]
0,"""llama2:7b""","""langchain_base""","""irrelevant_only""","""What is the primary function o…","""The mitochondria produce ATP a…","""The Eiffel Tower was completed…","[{""The Eiffel Tower was completed in 1889 and stands as a symbol of Paris, France."",""irrelevant""}, {""Solar panels convert sunlight into electricity using photovoltaic cells."",""irrelevant""}, … {""Basketball was invented by James Naismith in 1891."",""irrelevant""}]","""You are an assistant for quest…","""Use the following pieces of re…","""I don't know the primary funct…","[""I don't know"", ""The retrieved context does not contain information to answer the question""]"
0,"""llama2:7b""","""langchain_system""","""irrelevant_only""","""What is the primary function o…","""The mitochondria produce ATP a…","""The Eiffel Tower was completed…","[{""The Eiffel Tower was completed in 1889 and stands as a symbol of Paris, France."",""irrelevant""}, {""Solar panels convert sunlight into electricity using photovoltaic cells."",""irrelevant""}, … {""Basketball was invented by James Naismith in 1891."",""irrelevant""}]","""You are an assistant for quest…",null,"""I don't know the primary funct…","[""I don't know"", ""The retrieved context does not contain information to answer the question""]"
0,"""llama2:7b""","""langchain_user""","""irrelevant_only""","""What is the primary function o…","""The mitochondria produce ATP a…","""The Eiffel Tower was completed…","[{""The Eiffel Tower was completed in 1889 and stands as a symbol of Paris, France."",""irrelevant""}, {""Solar panels convert sunlight into electricity using photovoltaic cells."",""irrelevant""}, … {""Basketball was invented by James Naismith in 1891."",""irrelevant""}]",null,"""You are an assistant for quest…","""I don't know the primary funct…","[""I don't know"", ""The retrieved context does not contain information to answer the question""]"
0,"""llama2:7b""","""llamaindex_base""","""irrelevant_only""","""What is the primary function o…","""The mitochondria produce ATP a…","""The Eiffel Tower was completed…","[{""The Eiffel Tower was completed in 1889 and stands as a symbol of Paris, France."",""irrelevant""}, {""Solar panels convert sunlight into electricity using photovoltaic cells."",""irrelevant""}, … {""Basketball was invented by James Naismith in 1891."",""irrelevant""}]","""If you don't know the answer, …","""We have provided context infor…","""I don't know the answer to tha…","[""I don't know"", ""The retrieved context does not contain information to answer the question""]"
0,"""llama2:7b""","""llamaindex_system""","""irrelevant_only""","""What is the primary function o…","""The mitochondria produce ATP a…","""The Eiffel Tower was completed…","[{""The Eiffel Tower was completed in 1889 and stands as a symbol of Paris, France."",""irrelevant""}, {""Solar panels convert sunlight into electricity using photovoltaic cells."",""irrelevant""}, … {""Basketball was invented by James Naismith in 1891."",""irrelevant""}]","""If you don't know the answer, …",null,"""I don't know the answer to tha…","[""I don't know"", ""The retrieved context does not contain information to answer the question""]"
…,…,…,…,…,…,…,…,…,…,…,…
19,"""qwen3:4b""","""claude_rag_system""","""irrelevant_only""","""What device measures atmospher…","""A barometer measures atmospher…","""The Amazon River flows eastwar…","[{""The Amazon River flows eastward into the Atlantic Ocean."",""irrelevant""}, {""The moon orbits Earth approximately every 27 days."",""irrelevant""}, … {""Bicycles use gears to change mechanical advantage."",""irrelevant""}]","""Please remain faithful to the …",null,"""I don't know.""","[""I don't know"", ""The retrieved context does not contain information to answer the question""]"
19,"""qwen3:4b""","""cla

In [3]:
df["gold_answer"]

gold_answer
str
"""The mitochondria produce ATP a…"
"""Albert Einstein proposed the t…"
"""Plants absorb carbon dioxide d…"
"""The capital of Japan is Tokyo."""
"""The element with the symbol 'F…"
…
"""The fastest land animal is the…"
"""The Sahara Desert is located i…"
"""George Washington was the firs…"


In [4]:
df["model_answer"]

model_answer
str
"""Mitochondria are responsible f…"
"""Albert Einstein proposed the t…"
"""During photosynthesis, plants …"
"""Tokyo is the capital city of J…"
"""The chemical symbol 'Fe' stand…"
…
"""The cheetah is the fastest lan…"
"""The Sahara Desert is located o…"
"""George Washington was the firs…"


In [5]:
next(iter(df["gold_answer"]))

'The mitochondria produce ATP and serve as the cell’s primary source of energy.'

In [13]:
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

ans = next(iter(df["gold_answer"]))
resp = next(iter(df["model_answer"]))

In [7]:
scorer.score_multi(["A B C", "D E"], "A C")

{'rougeL': Score(precision=1.0, recall=0.6666666666666666, fmeasure=0.8)}

In [14]:
scorer.score(ans, resp)['rougeL'].precision

0.2727272727272727

In [15]:
scorer.score(ans, resp)['rougeL']

Score(precision=0.2727272727272727, recall=0.21428571428571427, fmeasure=0.23999999999999996)

In [16]:
scorer.score_multi([ans], resp)['rougeL']

Score(precision=0.2727272727272727, recall=0.21428571428571427, fmeasure=0.23999999999999996)

In [9]:
df["gold_answer"].len()

20

In [12]:
for ans, resp in zip(df["gold_answer"], df["model_answer"]):
    print(scorer.score(ans, resp))

{'rougeL': Score(precision=0.5833333333333334, recall=0.5, fmeasure=0.5384615384615384)}
{'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
{'rougeL': Score(precision=0.4444444444444444, recall=0.6666666666666666, fmeasure=0.5333333333333333)}
{'rougeL': Score(precision=0.5714285714285714, recall=0.6666666666666666, fmeasure=0.6153846153846153)}
{'rougeL': Score(precision=0.5714285714285714, recall=0.5, fmeasure=0.5333333333333333)}
{'rougeL': Score(precision=0.6363636363636364, recall=0.875, fmeasure=0.7368421052631579)}
{'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
{'rougeL': Score(precision=0.75, recall=1.0, fmeasure=0.8571428571428571)}
{'rougeL': Score(precision=0.3, recall=0.5, fmeasure=0.37499999999999994)}
{'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
{'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
{'rougeL': Score(precision=0.42857142857142855, recall=0.3333333333333333, fmeasure=0.375)}
{'rougeL': Score(precision=0.5833333333333334

In [10]:
for i in range(df["gold_answer"].len()):
    ans = df["gold_answer"][i]
    resp = df["model_answer"][i]
    print(i, scorer.score(ans, resp))

0 {'rougeL': Score(precision=0.5833333333333334, recall=0.5, fmeasure=0.5384615384615384)}
1 {'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
2 {'rougeL': Score(precision=0.4444444444444444, recall=0.6666666666666666, fmeasure=0.5333333333333333)}
3 {'rougeL': Score(precision=0.5714285714285714, recall=0.6666666666666666, fmeasure=0.6153846153846153)}
4 {'rougeL': Score(precision=0.5714285714285714, recall=0.5, fmeasure=0.5333333333333333)}
5 {'rougeL': Score(precision=0.6363636363636364, recall=0.875, fmeasure=0.7368421052631579)}
6 {'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
7 {'rougeL': Score(precision=0.75, recall=1.0, fmeasure=0.8571428571428571)}
8 {'rougeL': Score(precision=0.3, recall=0.5, fmeasure=0.37499999999999994)}
9 {'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
10 {'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
11 {'rougeL': Score(precision=0.42857142857142855, recall=0.3333333333333333, fmeasure=0.375)}
12 {'rougeL': Score

In [11]:
df[17].select(pl.col("gold_answer", "model_answer")).rows()

[('George Washington was the first U.S. president.',
  'George Washington was the first president of the United States.')]

In [18]:
from nltk.translate import bleu_score

ans = next(iter(df["gold_answer"]))
resp = next(iter(df["model_answer"]))
bleu_score.sentence_bleu([ans], resp)

0.48217931778851963

In [ ]:
bleu_score.sentence_bleu([ans], resp)

0.48217931778851963

In [19]:
for i in range(df["gold_answer"].len()):
    ans = df["gold_answer"][i]
    resp = df["model_answer"][i]
    print(i, bleu_score.sentence_bleu([ans], resp))

0 0.48217931778851963
1 1.0
2 0.7115535295670167
3 0.6736725254446679
4 0.5181323157300696
5 0.577497597217841
6 1.0
7 0.8136410108781797
8 0.47886173800789306
9 1.0
10 1.0
11 0.7317978729128142
12 0.5441546696164566
13 0.5865102576625455
14 1.0
15 0.9379094111501866
16 0.6491555850394412
17 0.6482123272483757
18 0.45028900034315333
19 0.5868292902204161


In [15]:
ans = df["gold_answer"][15]
resp = df["model_answer"][15]

print(scorer.score(ans, resp))
print(bleu_score.sentence_bleu([ans], resp))

{'rougeL': Score(precision=0.5714285714285714, recall=0.5714285714285714, fmeasure=0.5714285714285714)}
0.9379094111501866


In [16]:
df[15].select(pl.col("gold_answer", "model_answer")).rows()

[('The fastest land animal is the cheetah.',
  'The cheetah is the fastest land animal.')]

In [4]:
from bert_score import score as score_bert

In [5]:
ans = next(iter(df["gold_answer"]))
resp = next(iter(df["model_answer"]))

score_bert([resp], [ans], lang='en', verbose=True, use_fast_tokenizer=True)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 2.02 seconds, 0.50 sentences/sec


(tensor([0.8981]), tensor([0.9201]), tensor([0.9090]))

In [19]:
ans

'The mitochondria produce ATP and serve as the cell’s primary source of energy.'

In [20]:
resp

"Mitochondria are responsible for producing ATP, the cell's main energy currency."

In [12]:
testAns = "Abraham Lincoln was the 16th president of the United States"
testResp = "Abraham Lincoln was not the 16th president of the United States"

score_bert([testResp], [[testAns, "Abraham Lincoln wasn't the 16th president of the United States"]], rescale_with_baseline=True, lang='en', verbose=True, use_fast_tokenizer=True)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.03 seconds, 68.87 sentences/sec


(tensor([0.9593]), tensor([0.9593]), tensor([0.9594]))

In [ ]:
(tensor([0.8060]), tensor([0.8888]), tensor([0.8473]))
(tensor([0.9593]), tensor([0.9593]), tensor([0.9594]))


In [22]:
testAns = "Abraham Lincoln was the 16th president of the United States"
testResp = "Lincoln was the 16th president of the US"

score_bert([testAns], [testResp], rescale_with_baseline=True, lang='en', verbose=True, use_fast_tokenizer=True)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.02 seconds, 47.43 sentences/sec


(tensor([0.7910]), tensor([0.8656]), tensor([0.8283]))

In [23]:
testAns = "Abraham Lincoln was the 16th president of the United States"
testResp = "Lincoln was the 15th president of the US"

score_bert([testAns], [testResp], rescale_with_baseline=True, lang='en', verbose=True, use_fast_tokenizer=True)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.02 seconds, 49.47 sentences/sec


(tensor([0.7687]), tensor([0.8267]), tensor([0.7979]))

In [24]:
score_bert([testAns, "Howdy"], [testResp, "Hello"], rescale_with_baseline=True, lang='en', verbose=True, use_fast_tokenizer=True)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.03 seconds, 62.06 sentences/sec


(tensor([0.7687, 0.0674]), tensor([0.8267, 0.0707]), tensor([0.7979, 0.0705]))

In [25]:
df["model_answer"].to_list()

["Mitochondria are responsible for producing ATP, the cell's main energy currency.",
 'Albert Einstein proposed the theory of general relativity.',
 'During photosynthesis, plants absorb carbon dioxide from the air.',
 'Tokyo is the capital city of Japan.',
 "The chemical symbol 'Fe' stands for iron.",
 'The heart is responsible for pumping blood throughout the human body.',
 'Jupiter is the largest planet in the Solar System.',
 "William Shakespeare wrote the play 'Romeo and Juliet'.",
 'The freezing point of water in Celsius is 0 degrees.',
 'The Pacific Ocean is the largest ocean on Earth.',
 'Mars is known as the Red Planet.',
 'Leonardo da Vinci painted the Mona Lisa.',
 'DNA, or deoxyribonucleic acid, carries genetic information in nearly all living organisms.',
 'Paper was invented in ancient China during the Han Dynasty.',
 'The chemical formula for water is H2O.',
 'The cheetah is the fastest land animal.',
 'The Sahara Desert is located on the African continent.',
 'George Wa

In [ ]:
score_bert(df["gold_answer"].to_list(), df["model_answer"].to_list(), rescale_with_baseline=True, lang='en', verbose=True, use_fast_tokenizer=True)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.16 seconds, 122.61 sentences/sec


(tensor([0.6008, 1.0000, 0.8124, 0.7886, 0.6499, 0.8843, 1.0000, 0.9622, 0.6501,
         1.0000, 1.0000, 0.7281, 0.8850, 0.9572, 1.0000, 0.8587, 0.9351, 0.9156,
         0.8130, 0.8570]),
 tensor([0.6365, 1.0000, 0.6711, 0.7392, 0.6516, 0.7689, 1.0000, 0.8879, 0.4480,
         1.0000, 1.0000, 0.8138, 0.2658, 0.7732, 1.0000, 0.8850, 0.8602, 0.8138,
         0.6348, 0.6776]),
 tensor([0.6193, 1.0000, 0.7413, 0.7642, 0.6513, 0.8263, 1.0000, 0.9249, 0.5479,
         1.0000, 1.0000, 0.7710, 0.5587, 0.8640, 1.0000, 0.8720, 0.8975, 0.8644,
         0.7229, 0.7663]))